In [1]:
!pip install keras

In [2]:
!pip install tensorflow

In [3]:
import keras

Using TensorFlow backend.


In [4]:
import pandas as pd

df = pd.read_csv ('boneage-training-dataset.csv')
print (df)

          id  boneage   male
0       1377      180  False
1       1378       12  False
2       1379       94  False
3       1380      120   True
4       1381       82  False
5       1382      138   True
6       1383      150   True
7       1384      156   True
8       1385       36   True
9       1387      138   True
10      1388      126  False
11      1389      138   True
12      1390      156   True
13      1391      180   True
14      1393      132  False
15      1394       57   True
16      1395      120  False
17      1396      188  False
18      1398        4  False
19      1399       36  False
20      1400      132  False
21      1402       24  False
22      1403       60  False
23      1404      159  False
24      1405      120  False
25      1406      106  False
26      1407       30  False
27      1408      126   True
28      1409      149   True
29      1411       33  False
...      ...      ...    ...
12581  15577      144  False
12582  15578       72   True
12583  15579  

In [5]:
##Import libraries
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [6]:
IMAGE_SIZE = 64
# Base model with MobileNetV2
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

In [7]:
base_model = keras.applications.VGG16(input_shape=IMG_SHAPE,
                                               include_top=False, 
                                               weights='imagenet')

W0811 23:51:41.386970  5008 deprecation_wrapper.py:119] From C:\Users\Nanda Kumar\New folder\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0811 23:51:41.400958  5008 deprecation_wrapper.py:119] From C:\Users\Nanda Kumar\New folder\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0811 23:51:41.402952  5008 deprecation_wrapper.py:119] From C:\Users\Nanda Kumar\New folder\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0811 23:51:41.420909  5008 deprecation_wrapper.py:119] From C:\Users\Nanda Kumar\New folder\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0811 23:51:41.707175  5008 deprecation_wrapper.py:

In [8]:
base_model.trainable = False

In [9]:
# Trainable classification head
maxpool_layer = keras.layers.GlobalMaxPooling2D()
prediction_layer = keras.layers.Dense(1, activation='sigmoid')

In [10]:
#learning_rate = 0.0001

classifier = keras.Sequential([
    base_model,
    maxpool_layer,
    prediction_layer
])

In [11]:
classifier.compile(optimizer=keras.optimizers.Adam(),
             loss='binary_crossentropy',
             metrics=['accuracy'])

W0811 23:51:48.394038  5008 deprecation_wrapper.py:119] From C:\Users\Nanda Kumar\New folder\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0811 23:51:48.401019  5008 deprecation.py:323] From C:\Users\Nanda Kumar\New folder\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
## data importing and transforming and scaling
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [13]:
## Scaling test data
test_datagen = ImageDataGenerator(rescale=1./255)

In [16]:
train_set = train_datagen.flow_from_directory('boneage-training-dataset',
                                               target_size=(64, 64),
                                               batch_size=6,
                                               class_mode='binary')

Found 12611 images belonging to 1 classes.


In [15]:
test_set = test_datagen.flow_from_directory('boneage-test-dataset',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')


Found 200 images belonging to 1 classes.


In [20]:
label_map = (train_set.class_indices)
print(label_map)

{'boneage-training-dataset': 0}


In [21]:
## fitting model to images
classifier.fit_generator(
        train_set,
        steps_per_epoch=98,
        epochs=10,
        validation_data=test_set,
        validation_steps=38)

Epoch 1/10
98/98 [==============================] - 39s 403ms/step - loss: 0.0899 - acc: 0.9949 - val_loss: 0.0314 - val_acc: 1.0000
Epoch 2/10
98/98 [==============================] - 38s 389ms/step - loss: 0.0092 - acc: 1.0000 - val_loss: 0.0138 - val_acc: 1.0000
Epoch 3/10
98/98 [==============================] - 37s 382ms/step - loss: 0.0040 - acc: 1.0000 - val_loss: 0.0087 - val_acc: 1.0000
Epoch 4/10
98/98 [==============================] - 40s 406ms/step - loss: 0.0028 - acc: 1.0000 - val_loss: 0.0059 - val_acc: 1.0000
Epoch 5/10
98/98 [==============================] - 38s 391ms/step - loss: 0.0019 - acc: 1.0000 - val_loss: 0.0044 - val_acc: 1.0000
Epoch 6/10
98/98 [==============================] - 39s 400ms/step - loss: 0.0015 - acc: 1.0000 - val_loss: 0.0034 - val_acc: 1.0000s: 0.001
Epoch 7/10
98/98 [==============================] - 39s 403ms/step - loss: 0.0012 - acc: 1.0000 - val_loss: 0.0027 - val_acc: 1.0000
Epoch 8/10
98/98 [==============================] - 40s 405ms

In [22]:
## prediction of single new data
import numpy as np
import keras
from keras.preprocessing import image

In [24]:
test_image= image.load_img('boneage-training-dataset/boneage-training-dataset/10000.png',target_size =(64,64))

In [25]:
## Convert image to array
test_image = image.img_to_array(test_image)

In [26]:
## For single prediction change the dimension using axis. To remove problem of batch
test_image = np.expand_dims(test_image,axis = 0)
result = classifier.predict(test_image)

In [27]:
## Class of normal and abnormal

if result[0][0] == 1:
    prediction = 'True'
else:
   prediction = 'False'

In [28]:
prediction

'False'

In [29]:
test_image= image.load_img('boneage-test-dataset/boneage-test-dataset/4360.png',target_size =(64,64))

In [30]:
prediction

'False'